In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [1]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band1
  atmpro = mls
  band = 1
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0, 0]
  ng_refs = [3, 3, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (30, 250), (300, 250)]
  tsfc = 294
  vmax = 340
  vmin = 20
  w_diffuse = [(1.66, 1.66, 1.66), (1.66, 1.66, 1.66), (1.66, 1.66, 1.5, 1.5, 1.5, 1.5)]
  wgt = [(0.2, 0.2, 0.2), (0.2, 0.2, 0.2), (0.2, 0.2, 0.65, 0.65, 0.65, 0.65)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  5.484453                  NaN       NaN
0.000312      NaN       NaN                  1.0  3.200901
0.000750      2.0  4.371989                  2.0  3.027671
0.001052      3.0  3.964642                  3.0  2.947592
0.001476      4.0  3.533565                  4.0  2.826725
0.002070      5.0  3.096668                  5.0  2.655989
0.002904      6.0  2.673518                  6.0  2.429761
0.004074      7.0  2.278111                  7.0  2.149229
0.005714      8.0  1.917874                  8.0  1.827462
0.008015      9.0  1.594254                  9.0  1.492192
0.011243     10.0  1.306455                 10.0  1.183686
0.015771     11.0  1.053567                 11.0  0.938085
0.022122     12.0  0.833762                 12.0  0.762455
0.031031     13.0  0.641448                 13.0  0.630863
0.043528     14.0  0.465204                 14.0  0.500726
0.061057     15.0  0.373366                 15.0  0.415436
0.085645     16.0  0.371362                 16.0  0.387417
0.120136     17.0  0.376996                 17.0  0.361728
0.168516     18.0  0.375093                 18.0  0.342077
0.236378     19.0  0.379240                 19.0  0.348328
0.331549     20.0  0.395566                 20.0  0.377407
0.465100     21.0  0.433125                 21.0  0.420716
0.652400     22.0  0.498422                 22.0  0.480743
0.915100     23.0  0.516842                 23.0  0.507473
1.283650     24.0  0.496543                 24.0  0.515045
1.800600     25.0  0.479665                 25.0  0.524831
2.525700     26.0  0.452415                 26.0  0.494562
3.542800     27.0  0.427615                 27.0  0.458424
4.969550     28.0  0.405631                 28.0  0.413945
6.970850     29.0  0.383934                 29.0  0.370649
9.778100     30.0  0.361998                 30.0  0.348339
13.715850    31.0  0.339606                 31.0  0.334559
19.239350    32.0  0.316376                 32.0  0.308482
26.987250    33.0  0.292421                 33.0  0.288027
37.855300    34.0  0.270062                 34.0  0.269252
53.100050    35.0  0.228822                 35.0  0.222796
73.887500    36.0  0.160603                 36.0  0.149779
97.662500    37.0  0.101895                 37.0  0.085442
121.437500   38.0  0.111572                 38.0  0.091347
145.212500   39.0  0.164947                 39.0  0.144180
168.987500   40.0  0.218429                 40.0  0.198175
192.762500   41.0  0.346566                 41.0  0.323966
216.537500   42.0  0.702389                 42.0  0.677460
240.312500   43.0  1.040207                 43.0  1.013819
264.087500   44.0  1.251138                 44.0  1.282455
287.862500   45.0  1.300838                 45.0  1.346436
311.637500   46.0  1.226337                 46.0  1.222832
335.412500   47.0  1.093298                 47.0  1.145726
359.187500   48.0  0.932852                 48.0  0.981964
382.962500   49.0  0.738241                 49.0  0.736517
406.737500   50.0  0.577737                 50.0  0.580312
430.512500   51.0  0.434766                 51.0  0.457587
454.287500   52.0  0.309290                 52.0  0.321283
478.062500   53.0  0.205649                 53.0  0.190257
501.837500   54.0  0.127220                 54.0  0.095758
525.612500   55.0  0.074217                 55.0  0.044832
549.387500   56.0  0.042028                 56.0  0.022892
573.162500   57.0  0.024446                 57.0  0.014509
596.937500   58.0  0.015662                 58.0  0.011012
620.712500   59.0  0.012004                 59.0  0.009708
644.487500   60.0  0.012191                 60.0  0.010860
668.262500   61.0  0.016119                 61.0  0.015286
692.037500   62.0  0.021770                 62.0  0.021224
715.812500   63.0  0.022372                 63.0  0.022005
739.587500   64.0  0.014486                 64.0  0.014297
76

# Fluxes by Layer

CRD                           CLIRAD                \
                         flug       fldg      fnetg       flug          fldg   
pressure     level                                                             
0.000000e+00 1     -34.370336   0.000000 -34.370336        NaN           NaN   
1.000000e-08 1            NaN        NaN        NaN -34.159553  5.700803e-07   
6.244000e-04 2     -34.370353   0.000423 -34.369931 -34.159569  2.534918e-04   
8.759000e-04 3     -34.370360   0.000560 -34.369800 -34.159576  3.501300e-04   
1.228600e-03 4     -34.370370   0.000735 -34.369635 -34.159584  4.816997e-04   
1.723400e-03 5     -34.370381   0.000954 -34.369428 -34.159595  6.583699e-04   
2.417400e-03 6     -34.370396   0.001223 -34.369173 -34.159609  8.910476e-04   
3.390900e-03 7     -34.370416   0.001551 -34.368865 -34.159628  1.189949e-03   
4.756500e-03 8     -34.370441   0.001944 -34.368496 -34.159653  1.562097e-03   
6.672000e-03 9     -34.370473   0.002412 -34.368061 -34.159685  2.008924e-03   
9.358900e-03 10    -34.370516   0.002963 -34.367554 -34.159727  2.526446e-03   
1.312780e-02 11    -34.370574   0.003603 -34.366970 -34.159784  3.112070e-03   
1.841450e-02 12    -34.370652   0.004342 -34.366311 -34.159863  3.778144e-03   
2.583020e-02 13    -34.370764   0.005186 -34.365578 -34.159974  4.559371e-03   
3.623230e-02 14    -34.370932   0.006145 -34.364788 -34.160142  5.504020e-03   
5.082340e-02 15    -34.371225   0.007242 -34.363984 -34.160426  6.654420e-03   
7.129060e-02 16    -34.371692   0.008614 -34.363078 -34.160872  8.107395e-03   
1.000000e-01 17    -34.372301   0.010486 -34.361815 -34.161445  9.997875e-03   
1.402710e-01 18    -34.373033   0.013016 -34.360017 -34.162121  1.240007e-02   
1.967600e-01 19    -34.373896   0.016389 -34.357507 -34.162902  1.547022e-02   
2.759970e-01 20    -34.374880   0.020932 -34.353948 -34.163766  1.960411e-02   
3.871000e-01 21    -34.375919   0.027178 -34.348741 -34.164629  2.543428e-02   
5.431000e-01 22    -34.376727   0.035989 -34.340738 -34.165179  3.375931e-02   
7.617000e-01 23    -34.376334   0.048503 -34.327831 -34.164461  4.549208e-02   
1.068500e+00 24    -34.374250   0.065202 -34.309048 -34.161984  6.145939e-02   
1.498800e+00 25    -34.370666   0.086928 -34.283738 -34.157899  8.363055e-02   
2.102400e+00 26    -34.365195   0.115754 -34.249441 -34.151795  1.150557e-01   
2.949000e+00 27    -34.357799   0.153728 -34.204071 -34.143656  1.565191e-01   
4.136600e+00 28    -34.348117   0.204203 -34.143914 -34.133080  2.104412e-01   
5.802500e+00 29    -34.335450   0.271582 -34.063868 -34.119476  2.785326e-01   
8.139200e+00 30    -34.318946   0.361351 -33.957595 -34.101898  3.635607e-01   
1.141700e+01 31    -34.297622   0.480583 -33.817039 -34.079453  4.763818e-01   
1.601470e+01 32    -34.270434   0.638354 -33.632080 -34.051079  6.302381e-01   
2.246400e+01 33    -34.236403   0.846024 -33.390379 -34.015910  8.307623e-01   
3.151050e+01 34    -34.194319   1.117305 -33.077014 -33.972720  1.096261e+00   
4.420010e+01 35    -34.138627   1.467563 -32.671064 -33.916569  1.444884e+00   
6.200000e+01 36    -34.085660   1.897073 -32.188586 -33.865245  1.863378e+00   
8.577500e+01 37    -34.116820   2.380544 -31.736276 -33.904067  2.324070e+00   
1.095500e+02 38    -34.281788   2.832480 -31.449307 -34.080857  2.741517e+00   
1.333250e+02 39    -34.511724   3.376639 -31.135085 -34.321573  3.239521e+00   
1.571000e+02 40    -34.726650   4.056110 -30.670541 -34.544238  3.868285e+00   
1.808750e+02 41    -34.930961   4.875586 -30.055374 -34.754095  4.636324e+00   
2.046500e+02 42    -35.155978   6.076643 -29.079336 -34.985079  5.779794e+00   
2.284250e+02 43    -35.479707   8.378523 -27.101184 -35.317476  8.020331e+00   
2.522000e+02 44    -35.902802  11.731171 -24.171632 -35.754116  1.131250e+01   
2.759750e+02 45    -36.416625  15.768597 -20.648028 -36.284064  1.545463e+01   
2.997500e+02 46    -37.006561  20.022106 -16.984455 -36.891792  1.985474e+01   
3.235250e+02 47    -37.655

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')